In [511]:
import pandas as pd
import herepy
import gurobipy as gp
from gurobipy import GRB
import config
import datetime as dt
import numpy as np

In [512]:
geocoderApi = herepy.GeocoderApi(config.api_key)

# read in breweries
brewery_frame = pd.read_csv('brewery_list.csv')
# interpret open / close as times
coords= []
home_address = 'Calvary Episcopal Church, Pittsburgh, PA'
# home should be the first and last location, so add it to frame
home_frame = pd.DataFrame({'Name':['home'],
                          'Address': [home_address],
                          'Open': ['00:00'],
                          'Close': ['23:59'],
                          'Food': ['No']})
# add home_frame as the first and last row of brewery frame
brewery_frame = pd.concat([home_frame,brewery_frame,home_frame]).reset_index()
brewery_frame['Open'] = brewery_frame['Open'].apply(lambda x: dt.datetime.strptime(x,'%H:%M'))
brewery_frame['Close'] = brewery_frame['Close'].apply(lambda x: dt.datetime.strptime(x,'%H:%M'))
# create minutes after midnight columns in breweries frame for open and close
origin = brewery_frame.iloc[0]['Open']
brewery_frame['Open_minutes'] = brewery_frame['Open'].apply(lambda x: (x-origin).seconds/60)
brewery_frame['Close_minutes'] = brewery_frame['Close'].apply(lambda x: (x-origin).seconds/60)

In [558]:
coords = []

In [559]:
for i in range(0,brewery_frame.shape[0]):
    response = geocoderApi.free_form(brewery_frame.iloc[i]['Address'])
    try:
        position = response.as_dict()['items'][0]['access'][0]
    except:
        position = response.as_dict()['items'][0]['position']
    coord = ','.join([str(position['lat']), str(position['lng'])])
    coords.append(coord)
brewery_frame['coords'] = coords

In [523]:
# find all pairwise route times, store in a matrix by iterating through rows twice
routing = herepy.RoutingApi(config.api_key)
distances = list()
for i in range(0,brewery_frame.shape[0]):
    distance_subset = list()
    start = brewery_frame.iloc[i]['Address']
    for j in range(0,brewery_frame.shape[0]):
        end = brewery_frame.iloc[j]['Address']
        route = routing.pedastrian_route(start,end)
        distance_subset.append(route.as_dict()['response']['route'][0]['leg'][0]['travelTime']/60)
    distances.append(distance_subset)

array([60, 60, 45, 60, 60, 45, 60, 45, 45, 45, 45, 60, 45, 45, 60, 45, 60,
       45, 45, 45, 45, 60, 60])

In [672]:
# define number of breweries we want to force our model to visit, start with 5 to encourage feasibility
# note that slots includes both home and away
# define brewery goal
brewery_goal = 9
slots = range(brewery_goal+2)
# define stay length in minutes at each brewery
stay_length = 45
# define the number of stops
stops = range(brewery_frame.shape[0])
# create model
m = gp.Model('breweries')
# add decision variables - i x s matrix indicating if location i visited in slot s
theta = m.addVars(stops, slots, vtype=GRB.BINARY, name = 'theta_is')
# add decision variables for whether or not route is take from i to j
X = m.addVars(stops,stops, vtype=GRB.BINARY, name = 'x_ij')
# add a decision variable for Ts - Time of arrival at slot S in midnights after midnight
T = m.addVars(slots,vtype=GRB.INTEGER)
# define objective
m.setObjective(sum(sum(distances[i][j]*X[i,j] for i in stops) for j in stops),
              GRB.MINIMIZE)

In [673]:
# add a constraint forcing T[s] to take the right value ####################################################################################
for s in slots[1:]:
    m.addConstr(T[s],GRB.GREATER_EQUAL,
               #30*(s-1)+sum(sum(sum(int(distances[i][j])*X[i,j]*theta[i,n] for i in stops) for j in stops) for n in slots[0:s-1]))
                stay_length+T[s-1]+sum(sum(distances[i][j]*X[i,j]*theta[i,s-1] for i in stops)for j in stops))
# add constraint limiting X[i,j] to only turn on if theta[i,s] = 1 and theta[j,s+1] = 1
for i in stops:
    for j in stops:
            m.addConstr(X[i,j],GRB.EQUAL,sum(theta[i,s]*theta[j,s+1] for s in slots[0:-1]))
# add constraint forcing one slot per brewery max
for i in stops:
    m.addConstr(sum(theta[i,s] for s in slots),GRB.LESS_EQUAL,1)
# add constraint forcing one brewery per slot
for s in slots:
    m.addConstr(sum(theta[i,s] for i in stops),GRB.EQUAL,1)
# force first and last slot to be home
m.addConstr(theta[0,0],GRB.EQUAL,1)
m.addConstr(theta[stops[-1],slots[-1]],GRB.EQUAL,1)

<gurobi.Constr *Awaiting Model Update*>

In [674]:
# enforce opening and closing hours
for s in slots:
    for i in stops:
        m.addConstr(T[s],GRB.GREATER_EQUAL,brewery_frame['Open_minutes'][i]*theta[i,s])
        m.addConstr((T[s]+stay_length)*theta[i,s],GRB.LESS_EQUAL,
               brewery_frame['Close_minutes'][i])
# cap T at the number of minutes in a day
for s in slots:
    m.addConstr(T[s],GRB.LESS_EQUAL,24*60-1)

In [675]:
m.Params.InfUnbdInfo=1
m.optimize()

Changed value of parameter InfUnbdInfo to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 300 rows, 793 columns and 1003 nonzeros
Model fingerprint: 0x29d62ece
Model has 792 quadratic constraints
Variable types: 0 continuous, 793 integer (782 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+02]
  QMatrix range    [1e+00, 3e+02]
  QLMatrix range   [1e+00, 4e+01]
  Objective range  [4e+00, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
  QRHS range       [4e+01, 1e+03]
Presolve removed 80 rows and 154 columns
Presolve time: 0.03s
Presolved: 11407 rows, 4248 columns, 29847 nonzeros
Presolved model has 9 quadratic constraint(s)
Variable types: 0 continuous, 4248 integer (4158 binary)

Root relaxation: objective 3.965000e+01, 113 iterations, 0.04 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd

 28045  4767     cutoff   63       282.23333  203.00000  28.1%  40.1  470s
 28865  4927  253.51667   65   40  282.23333  204.18081  27.7%  40.2  476s
 29348  5088  250.14444   68   50  282.23333  204.64667  27.5%  40.3  491s
 29830  5157     cutoff   54       282.23333  205.17299  27.3%  40.3  495s
 30365  5313  265.98519   67   62  282.23333  205.63750  27.1%  40.3  500s
 30767  5425  265.84167   65   36  282.23333  206.13875  27.0%  40.4  506s
 31257  5517  227.81500   78   37  282.23333  206.78333  26.7%  40.5  512s
 31768  5633  265.08750   74   33  282.23333  207.56667  26.5%  40.6  516s
 32342  5738  236.72237   58   46  282.23333  208.22417  26.2%  40.6  520s
 32819  5846     cutoff   86       282.23333  208.60000  26.1%  40.6  525s
 33563  6082  253.44444   63   19  282.23333  209.39833  25.8%  40.6  531s
 34115  6149  233.07405   75   81  282.23333  209.92586  25.6%  40.6  535s
 34934  6327  282.15590   70   44  282.23333  210.85861  25.3%  40.5  542s
 35451  6420  238.75000  

In [677]:
# parse out decision
included = []
arrival_times = []
for s in slots:
    #print('slot:' +str(s))
    for i in stops:
        if round(theta[i,s].x) ==1.0:
            #print(brewery_frame.iloc[i]['Name'])
            #print((origin+pd.Timedelta(minutes=T[s].x)).time())
            included.append(i)
            arrival_times.append((origin+pd.Timedelta(minutes=T[s].x)))
itinerary = brewery_frame.iloc[included][['Name', 'Address', 'Open', 'Close','Food']].reset_index(drop=True)
# assign arrival Time
itinerary['Arrival Time'] = arrival_times
# calculate departure time
itinerary['Departure Time'] = itinerary['Arrival Time'].apply(lambda x: x+pd.Timedelta(minutes=stay_length))
itinerary['Open'] = itinerary['Open'].apply(lambda x: x.time().strftime('%H:%M'))
itinerary['Close'] = itinerary['Close'].apply(lambda x: x.time().strftime('%H:%M'))
rows = itinerary.shape[0]
# grab minutes to next destination
distance_to_next = []
for k in slots[0:-1]:
    i = included[k]
    j = included[k+1]
    distance_to_next.append(round(distances[i][j]))
# departure time from home = arrival second destination - walk to first destination
depart_home = itinerary.iloc[1]['Arrival Time'] - pd.Timedelta(minutes = distance_to_next[0])
arrive_home = itinerary.iloc[rows-2]['Departure Time'] + pd.Timedelta(minutes = distance_to_next[-1])
itinerary.loc[0,'Departure Time'] = depart_home
itinerary.loc[rows-1,'Arrival Time'] = arrive_home
# now that calculations on time are down, strip unneeded date information
itinerary['Arrival Time'] = itinerary['Arrival Time'].apply(lambda x: x.time().strftime('%H:%M'))
itinerary['Departure Time'] = itinerary['Departure Time'].apply(lambda x: x.time().strftime('%H:%M'))
itinerary.loc[0,'Arrival Time'] = '--'
itinerary.loc[rows-1,'Departure Time'] = '--'
itinerary.loc[0,'Open'] = '--'
itinerary.loc[0,'Close'] = '--'
itinerary.loc[0,'Food'] = '--'
itinerary.loc[rows-1,'Open'] = '--'
itinerary.loc[rows-1,'Close'] = '--'
itinerary.loc[rows-1,'Food'] = '--'
# strip everything after comma for address
itinerary['Address'] = itinerary['Address'].str.replace(',.*','')
# make distance to next string
distance_to_next_str = [str(x) for x in distance_to_next]
# add in -- for last one
distance_to_next_str.append('--')
itinerary['Minutes to next stop'] = distance_to_next_str
# fix the first and last arrival time since the model doesn't care about those
itinerary.reset_index(drop=True)

,Name,Address,Open,Close,Food,Arrival Time,Departure Time,Minutes to next stop
0,home,Calvary Episcopal Church,--,--,--,--,11:32,28
1,East End,147 Julius St,12:00,18:00,Yes,12:00,12:45,19
2,Brewdog,6144 Centre Ave,11:00,22:00,Yes,13:05,13:50,50
3,Trace,4312 Main St,13:00,22:00,Yes,14:40,15:25,17
4,Church,3525 Liberty Ave,12:00,23:00,Yes,15:43,16:28,41
5,Helltown,1700 Penn Ave,11:00,23:00,No,17:09,17:54,4
6,Insurrection,1812 Penn Ave,11:00,21:00,No,17:59,18:44,15
7,Cinderlands,2601 Smallman St,11:30,23:00,Yes,18:59,19:44,30
8,Eleventh Hour,3711 Charlotte St,13:00,21:00,Yes,20:15,21:00,4
9,Burghers,3601 Butler St,11:00,22:00,Yes,21:14,21:59,74


In [476]:
itinerary.iloc[1]['Open'].strftime('%H:%M')

'12:00'

In [683]:
itinerary.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Name</th>\n      <th>Address</th>\n      <th>Open</th>\n      <th>Close</th>\n      <th>Food</th>\n      <th>Arrival Time</th>\n      <th>Departure Time</th>\n      <th>Minutes to next stop</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>home</td>\n      <td>Calvary Episcopal Church</td>\n      <td>--</td>\n      <td>--</td>\n      <td>--</td>\n      <td>--</td>\n      <td>11:32</td>\n      <td>28</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>East End</td>\n      <td>147 Julius St</td>\n      <td>12:00</td>\n      <td>18:00</td>\n      <td>Yes</td>\n      <td>12:00</td>\n      <td>12:45</td>\n      <td>19</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Brewdog</td>\n      <td>6144 Centre Ave</td>\n      <td>11:00</td>\n      <td>22:00</td>\n      <td>Yes</td>\n      <td>13:05</td>\n      <td>13:50</td>\n      <td>50</td>\n    </

In [607]:
12+65+4+30+15+4+41+17+57

245

In [ ]:
sum(itinerary[''])

In [679]:
# create an api call
main_string = 'https://image.maps.ls.hereapi.com/mia/1.6/routing?apiKey='+config.api_key+'&TransportModeType=pedestrian'
# create a string that joins all of the waypoints together
included_coords = list(brewery_frame.loc[included]['coords'])
waypoint_list = []
poix_list = []
for i in range(len(included_coords)):
    waypoint_list.append('waypoint'+str(i)+'='+str(included_coords[i]))
    poix_list.append('poix'+str(i)+'='+str(included_coords[i]))

In [680]:
# make a separate map of all brewery optio

In [681]:
api_call = main_string+'&'+('&'.join(waypoint_list))+'&'+('&'.join(poix_list))
import requests
from io import StringIO
requests.get(api_call)


<Response [200]>

In [682]:
api_call

'https://image.maps.ls.hereapi.com/mia/1.6/routing?apiKey=-eGe5WpGhIl5depjfbvcLDasVOx64obHjD_CPfoO3Wk&TransportModeType=pedestrian&waypoint0=40.45652,-79.92217&waypoint1=40.45919,-79.91144&waypoint2=40.46051,-79.92303&waypoint3=40.46424,-79.9533&waypoint4=40.46183,-79.96429&waypoint5=40.44983,-79.98587&waypoint6=40.45064,-79.9846&waypoint7=40.45573,-79.97906&waypoint8=40.46689,-79.96541&waypoint9=40.46548,-79.96524&waypoint10=40.45652,-79.92217&poix0=40.45652,-79.92217&poix1=40.45919,-79.91144&poix2=40.46051,-79.92303&poix3=40.46424,-79.9533&poix4=40.46183,-79.96429&poix5=40.44983,-79.98587&poix6=40.45064,-79.9846&poix7=40.45573,-79.97906&poix8=40.46689,-79.96541&poix9=40.46548,-79.96524&poix10=40.45652,-79.92217'

In [656]:
# map oout all breweries for blog post
import folium
icon_size = (14, 14)
lon, lat =  -79.9532, 40.4443 
zoom_start = 12
mapa = folium.Map(location=[lat, lon], tiles="Cartodb Positron",
                  zoom_start=zoom_start)
# loop through brewery_frame coords, define lat lon
coord_for_map = []
for i in range(brewery_frame.shape[0]):
    coordinates = brewery_frame.iloc[i]['coords'].split(',')
    icon_url = 'https://devrajkori.com/wp-content/uploads/2021/04/beer_icon.png'
    icon = folium.features.CustomIcon(icon_url,
                                  icon_size=(14, 14))
    # make the name a clickable popup
    popup = brewery_frame.iloc[i]['Name']
    marker = folium.map.Marker(coordinates, icon=icon, popup=folium.map.Popup(popup))
    mapa.add_children(marker)
mapa

/Users/devrajkori/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: FutureWarning: Method `add_children` is deprecated. Please use `add_child` instead.


In [657]:
mapa.save('all_breweries_map.html')

In [640]:
api_call2

'https://image.maps.ls.hereapi.com/mia/1.6/routing?apiKey=-eGe5WpGhIl5depjfbvcLDasVOx64obHjD_CPfoO3Wk&poix0=40.45652,-79.92217&poix1=40.45446,-80.00003&poix2=40.45697,-79.99178&poix3=40.45064,-79.9846&poix4=40.44983,-79.98587&poix5=40.49457,-79.93265&poix6=40.4963,-79.92657&poix7=40.46689,-79.96541&poix8=40.46548,-79.96524&poix9=40.4065,-79.90998&poix10=40.48444,-79.94795'

In [641]:
main_string

'https://image.maps.ls.hereapi.com/mia/1.6/routing?apiKey=-eGe5WpGhIl5depjfbvcLDasVOx64obHjD_CPfoO3Wk'

In [658]:
mapa